<h1> Downscaling Landcover data using Cloud ML </h1>

In this notebook, we train a TensorFlow model to fit Landsat 8 bands to a low-resolution landcover map. Then, we use that model on the high-resolution Landsat data to create a high-resolution landcover map. In essence, we are using TensorFlow to "statistically downscale" the landcover data (note that the term "downscaling" is counterintuitive -- downscaling an image increases its resolution or upsamples it).

<br/>
<h3> Set up </h3>

As a first step, we install a Python package capable of reading GDAL files.

In [2]:
# Note: after running this cell, you need to Reset Session in Datalab to pick up the new package
# You may need to change this to "!sudo apt-get" if you get permission problems.
!apt-get -y install python-gdal

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-gdal is already the newest version.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


The GDAL library won't be able to read directly from Cloud Storage, so let's download the data to disk. Note that if you started Datalab on a VM without enough space, this might fail.  If you launched Datalab using the codelab instructors, change instance_details.sh to have a disk of at least 20 GB. 

In [1]:
%bash
if [ ! -d input_data ]; then
   mkdir input_data
   for band in `seq 1 7`; do
     gsutil cp gs://mdh-test/landsat-ml/landsat8-b$band.tif input_data
   done
   gsutil cp gs://mdh-test/landsat-ml/srtm-elevation.tif input_data
   gsutil cp gs://mdh-test/landsat-ml/mcd12-labels.tif input_data
fi

Copying gs://mdh-test/landsat-ml/landsat8-b1.tif...
| [1 files][  1.0 GiB/  1.0 GiB]   77.7 MiB/s                                   
Operation completed over 1 objects/1.0 GiB.                                      
Copying gs://mdh-test/landsat-ml/landsat8-b2.tif...
/
Operation completed over 1 objects/1.1 GiB.                                      
Copying gs://mdh-test/landsat-ml/landsat8-b3.tif...
- [1 files][  1.1 GiB/  1.1 GiB]   66.8 MiB/s                                   
Operation completed over 1 objects/1.1 GiB.                                      
Copying gs://mdh-test/landsat-ml/landsat8-b4.tif...
- [1 files][  1.1 GiB/  1.1 GiB]   72.5 MiB/s                                   
Operation completed over 1 objects/1.1 GiB.                                      
Copying gs://mdh-test/landsat-ml/landsat8-b5.tif...
- [1 files][  1.1 GiB/  1.1 GiB]   79.4 MiB/s                                   
Operation completed over 1 objects/1.1 GiB.                                      
Copy

<h3> Preprocessing </h3>

Preprocessing in Cloud ML is done using Cloud Dataflow.  We'll need to read the Geotiffs and then merge them in such a way that all the data corresponding to a pixel becomes a single TFRecord.

In [2]:
import osgeo.gdal as gdal
import struct
import numpy as np

def read_geotiff(filename):
    """
    return 1-D array of pixel values
    """
    ds = gdal.Open( filename, gdal.GA_ReadOnly )
    band = ds.GetRasterBand(1)
    ncols = band.XSize
    nrows = band.YSize
    print "Reading ", nrows, "x", ncols, " image from ", filename
    packformat = 'f' * ncols
    result = np.zeros([nrows*ncols], dtype=np.float32)  # 1-D array of pixel values
    for line in xrange(0, nrows):
      line_data = struct.unpack(packformat, band.ReadRaster(0, line, ncols, 1, ncols, 1, gdal.GDT_Float32))
      result[line*ncols:(line+1)*ncols] = line_data
    ds = None
    return result

In [ ]:
b1 = read_geotiff('input_data/landsat8-b1.tif')
print b1[200]

Reading  16384 x 16384  image from  input_data/landsat8-b1.tif


In [5]:
b2 = read_geotiff('input_data/landsat8-b2.tif')
print b2[200]

Reading  16384 x 16384  image from  input_data/landsat8-b2.tif
0.0868123


In [6]:
import google.cloud.ml as ml
import tensorflow as tf
print tf.__version__
print ml.sdk_location
import google.cloud.ml.features as features

#import google.cloud.ml as ml
#print ml.sdk_location

class LandcoverFeatures(object):
  target = features.target('landcover').discrete()  # classification problem
  inputs = [
      features.numeric('b1').identity(),
      features.numeric('b2').identity(),
      #features.numeric('b3').identity(),
      #features.numeric('b4').identity(),
      #features.numeric('b5').identity(),
      #features.numeric('b6').identity(),
      #features.numeric('b7').identity(),
      #features.numeric('el').identity(),  # elevation
  ]

0.11.0rc0
gs://cloud-ml/sdk/cloudml-0.1.6-alpha.dataflow.tar.gz


In [ ]:
import apache_beam as beam
import google.cloud.ml as ml
import google.cloud.ml.io as io
import os

# defines
feature_set = LandcoverFeatures()
OUTPUT_DIR = './preproc'
pipeline = beam.Pipeline('DirectPipelineRunner')

# preprocessing
training_data = beam.Create([])
train = pipeline | beam.Read('ReadTrainingData', training_data)
eval_data = beam.io.TextFileSource(
    '../lab1a/taxi-valid.csv',
    strip_trailing_newlines=True,
    coder=io.CsvCoder.from_feature_set(feature_set, feature_set.csv_columns))
eval = pipeline | beam.Read('ReadEvalData', eval_data)

(metadata, train_features, eval_features) = ((train, eval) |
   'Preprocess' >> ml.Preprocess(feature_set))

(metadata
   | 'SaveMetadata'
   >> io.SaveMetadata(os.path.join(OUTPUT_DIR, 'metadata.yaml')))
(train_features
   | 'WriteTraining'
   >> io.SaveFeatures(os.path.join(OUTPUT_DIR, 'features_train')))
(eval_features
   | 'WriteEval'
   >> io.SaveFeatures(os.path.join(OUTPUT_DIR, 'features_eval')))

# run pipeline
pipeline.run()